## MTA API Interface / Parsing
The code in the below cell simply interfaces with the MTA API and parses the response into a reasonable DataFrame object.

In [1]:
import requests
import pandas as pd
import numpy as np
from datetime import datetime

# defining constants for ease-of-use
lng_key = "MonitoredVehicleJourney_VehicleLocation_Longitude"
lat_key = "MonitoredVehicleJourney_VehicleLocation_Latitude"
bearing_key="MonitoredVehicleJourney_Bearing"
direction_key="MonitoredVehicleJourney_DirectionRef"
progress_key='MonitoredVehicleJourney_ProgressRate'
line_key=u'MonitoredVehicleJourney_LineRef'
dist_from_base_key = "MonitoredVehicleJourney_MonitoredCall_Extensions_Distances_CallDistanceAlongRoute"
dist_to_next_stop_key = u'MonitoredVehicleJourney_MonitoredCall_Extensions_Distances_DistanceFromCall'
timestamp_key = "RecordedAtTime"
join_key = "MonitoredVehicleJourney_FramedVehicleJourneyRef_DatedVehicleJourneyRef"

MTA_API_KEY=...
MTA_API_BASE = "http://bustime.mta.info/api/siri/vehicle-monitoring.json"
def _flatten_dict(root_key, nested_dict, flattened_dict):
    for key, value in nested_dict.iteritems():
        next_key = root_key + "_" + key if root_key != "" else key
        if isinstance(value, dict):
            _flatten_dict(next_key, value, flattened_dict)
        else:
            flattened_dict[next_key] = value
    return flattened_dict
    
#This is useful for the live MTA Data
params = {"key": MTA_API_KEY, }
BUS_ID="MTA NYCT_M116"
params["LineRef"] = BUS_ID
def nyc_current():
    resp = requests.get(MTA_API_BASE, params=params).json()
    info = resp['Siri']['ServiceDelivery']['VehicleMonitoringDelivery'][0]['VehicleActivity']
    return pd.DataFrame([_flatten_dict('', i, {}) for i in info])

## Iterative Update Code

Here, we implement the following kalman filter:
- Our hidden state is the 4-element vector [lat, lng, velocity_lat, velocity_lng]\*
- Our measured state is [lat, lng, v_lat, v_lng] (see below: we could measure a velocity based on our previous state)
- Our state transformation matrix is lat_new := dt * velocity_lat * current_lat (and likewise for lng). Velocities stay constant
- Control matrix is just the identity. 
- Observation matrix is the identity.

A couple of important notes:
- We currently "measure" velocity at 5.5mph. A better approach would be to take the average velocity from the previous two timesteps. An even better approach would be to smooth this average with a prior derived from historical data. 5.5mph seems to work well enough for a first stab, though. An alternative approach would be to not measure velocity, since it's an ugly thing to measure. 
- An alternative, and probably comparable approach, would be to let the filter infer velocity as a non-measured state. We could do this by the following modification: 
    - Our measured state is (lat, lng, lat_old, lng_old)
    - Observation matrix is: lat_est := lat (likewise for lng), v_lat_est := (lat - lat_old) / dt (likewise for lng)

- Notably, velocity is also likely not linear: I'd expect it's sinusoidal with time (or with distance to next intersection). We'd want to improve our modelling there too.
- We could plug in a substantial amount of information into our control matrix. For example, on manhattan bus lines, we could set up features to compute if the bus was going to need to make a 90 degree turn based on current position and next stop along the route.
- (\* from above) We're assuming our lat/long coordinates are locally cartesian enough to make estimates without worrying about spherical projections or nonlinear distance formulae. Again, incorporating better distance measurements is a v2 feature (and easily doable with a nonlinear kalman filter)


In [27]:
import geopy
from geopy import Point
from geopy.distance import vincenty
from geopy.distance import VincentyDistance
from collections import namedtuple

KFTuple = namedtuple("KFTuple", ["model", "states", "means", "covariances", "update_times", "errors"])
KFTuple.__new__.__defaults__ = (None,) * len(KFTuple._fields)
time_step = 5 # in seconds

measurements_by_ref = {} # table of bus positions by route ID
def update_route_measurements(cur_data):
    route_refs = []
    for row in cur_data.iterrows():
        info = row[1]
        route_ref = info[join_key]
        route_refs.append(route_ref)
        if route_ref not in measurements_by_ref:
            measurements_by_ref[route_ref] = KFTuple(states=[info])
        if measurements_by_ref[route_ref].states[-1][timestamp_key] == info[timestamp_key]:
            continue # we're at same measurement
      
        measurements_by_ref[route_ref].states.append(info)
    
    return "OK"

def update_bus_info():
    cur_info =  nyc_current()
    cur_info[timestamp_key] = cur_info[timestamp_key].apply(iso_to_datetime)
    update_route_measurements(cur_info)
    for route, kf_tuple in measurements_by_ref.iteritems():
        measurements_by_ref[route] = update_model(kf_tuple)

def iso_to_datetime(ts_string):
    return datetime.strptime(ts_string.split(".")[0], "%Y-%m-%dT%H:%M:%S")

        
from pykalman import KalmanFilter
def init_model(kf_tuple):
    init = kf_tuple.states[-1]
    # TODO: seed average bus speed based on model
    v_lat, v_lng = v_to_components(5.5 / 3600., # assume bus is going 5.5mph to start [avg line speed]
                                   init[bearing_key],
                                   init[lat_key],
                                   init[lng_key])
    init['v_lat'] = v_lat
    init['v_lng'] = v_lng
    initial_state = kf_tuple.states[-1][[lat_key, 
                                        lng_key, 
                                        "v_lat",
                                        "v_lng"]]
    model = KalmanFilter(initial_state_mean=initial_state,
                         initial_state_covariance=np.eye(4),
                         transition_matrices=transition_matrix(5),
                         n_dim_obs=4)
    return KFTuple(model, kf_tuple.states, [initial_state], [np.eye(4)], [init[timestamp_key]], [])

def update_model(kf_tuple):
    model = kf_tuple.model 
    
    if model is None:
        return init_model(kf_tuple)
    
    latest = kf_tuple.states[-1]
    # nothing to do
    if latest[timestamp_key] == kf_tuple.update_times[-1]:
        mean, cov = kf_tuple.model.filter_update(kf_tuple.means[-1],
                                                 kf_tuple.covariances[-1])
        kf_tuple.means.append(mean)
        kf_tuple.covariances.append(cov)
        return kf_tuple
    else:
        cur = kf_tuple.states[-1]
        v_lat, v_lng = v_to_components(v_estimate(kf_tuple.states), # average bus speed, miles per second
                                   cur[bearing_key],
                                   cur[lat_key],
                                   cur[lng_key])
        cur['v_lat'] = v_lat
        cur['v_lng'] = v_lng
        cur_state = kf_tuple.states[-1][[lat_key, 
                                        lng_key, 
                                        "v_lat",
                                        "v_lng"]]
        mean, cov = kf_tuple.model.filter_update(kf_tuple.means[-1],
                                                 kf_tuple.covariances[-1],
                                                 cur_state)
        interp_mean, interp_cov = kf_tuple.model.filter_update(kf_tuple.means[-1],
                                                 kf_tuple.covariances[-1])
        
        kf_tuple.errors.append(forecast_error(mean, interp_mean))
        kf_tuple.means.append(mean)
        kf_tuple.covariances.append(cov)
        kf_tuple.update_times.append(latest[timestamp_key])
        return kf_tuple

    
import numpy as np
def transition_matrix(dt):
    T = np.eye(4)
    T[0][2] = dt
    T[1][3] = dt
    return T

from geopy.distance import vincenty
def v_estimate(states):
    # average bus speed
    if len(states) == 1:
        return 5.5 / 3600.
    else:
        p1 = (states[-2][lat_key], states[-2][lng_key])
        p2 = (states[-1][lat_key], states[-1][lng_key])
        t1 = states[-2][timestamp_key]
        t2 = states[-1][timestamp_key]
        return vincenty(p1, p2).miles / float((t2 - t1).seconds)

def forecast_error(mean, interp_mean):
    p1 = tuple(mean[:2])
    p2 = tuple(interp_mean[:2])
    return vincenty(p1, p2).miles

# convert v, bearing to v_x, v_y components
def v_to_components(v, bearing, lat, lng):
    d = VincentyDistance(miles=v) # num miles / second
    res = d.destination(geopy.Point(lat, lng), 90 - bearing) # nyc uses east / counterclockwise convention
    out = (res.latitude - lat, res.longitude - lng)
    return out


def run():
    import time
    for i in range(50):
        update_bus_info()
        time.sleep(time_step)
        print "iteration {}".format(i)


In [76]:
import dill
with open('measurements3.pkl', 'w+') as out_f:
    dill.dump(measurements_by_ref, out_f)

In [79]:
print "\n".join([str((k, len(v.states))) for k, v in measurements_by_ref.iteritems()])
route_id="MV_A5-Weekday-SDon-114500_M116_316"

('MV_A5-Weekday-SDon-111500_M116_314', 13)
('MV_A5-Weekday-SDon-115500_M116_314', 1)
('MV_A5-Weekday-SDon-111800_M116_312', 18)
('MV_A5-Weekday-SDon-113500_M116_317', 3)
('MV_A5-Weekday-SDon-112500_M116_313', 14)
('MV_A5-Weekday-SDon-110900_M116_315', 19)
('MV_A5-Weekday-SDon-110500_M116_316', 3)
('MV_A5-Weekday-SDon-114500_M116_316', 22)
('MV_A5-Weekday-SDon-109500_M116_317', 13)


In [4]:
import ujson as json
def kftuple_to_json(kf_tuple):
    output_dict = {"actual": [(x[timestamp_key], x[lat_key], x[lng_key], x[bearing_key]) for x in kf_tuple.states],
                   "preds": [(m, c) for m, c in zip(kf_tuple.means, zip(kf_tuple.covariances))]}
    return json.dumps(output_dict)

In [90]:
# kftuple_to_json(measurements_by_ref[route_id])

In [80]:
f = open("visualization/static/test.json", "w+")
f.write(kftuple_to_json(measurements_by_ref[route_id]))
f.close()

In [104]:
kf_tuple = measurements_by_ref[route_id]
[(x[lat_key], x[lng_key]) for x in kf_tuple.states]
# [(x[0], x[1], x[2], x[3]) for x in kf_tuple.means]

[(40.800321, -73.965437),
 (40.79968, -73.963916),
 (40.799523, -73.963545),
 (40.799147, -73.962651),
 (40.798861, -73.96197)]

In [106]:
for i, t in measurements_by_ref.iteritems():
    print "========{}======".format(i)
    print "\n".join([str((x[lat_key], x[lng_key])) for x in t.states])

========MTA NYCT_MV_A7-Weekday-SDon-091600_M116_208======
(40.800915, -73.959823)
(40.800711, -73.959972)
(40.80026, -73.960303)
(40.798793, -73.961375)
(40.798793, -73.961375)
(40.799139, -73.96263)
(40.799673, -73.963899)
(40.800363, -73.965537)
(40.800346, -73.965496)
========MTA NYCT_MV_A7-Weekday-SDon-089600_M116_206======
(40.797466, -73.930856)
(40.797466, -73.930856)
(40.797466, -73.930856)
(40.797466, -73.930856)
(40.797466, -73.930856)
(40.797466, -73.930856)
(40.797509, -73.930956)
(40.797551, -73.931056)
========MTA NYCT_MV_A7-Weekday-SDon-090300_M10_9======
(40.797381, -73.930656)
(40.797381, -73.930656)
(40.797381, -73.930656)
(40.797381, -73.930656)
(40.797381, -73.930656)
(40.797381, -73.930656)
(40.797381, -73.930656)
(40.797381, -73.930656)
(40.797381, -73.930656)
========MTA NYCT_MV_A7-Weekday-SDon-093500_M10_5======
(40.800321, -73.965437)
(40.79968, -73.963916)
(40.799523, -73.963545)
(40.799147, -73.962651)
(40.798861, -73.96197)
========MTA NYCT_MV_A7-Weekday-SDo

In [7]:
def set_live_keys():
    to_update = {
        "lng_key":"MonitoredVehicleJourney_VehicleLocation_Longitude",
        "lat_key": "MonitoredVehicleJourney_VehicleLocation_Latitude",
        "bearing_key":"MonitoredVehicleJourney_Bearing",
        "direction_key":"MonitoredVehicleJourney_DirectionRef",
        "progress_key":'MonitoredVehicleJourney_ProgressRate',
        "line_key":u'MonitoredVehicleJourney_LineRef',
        "dist_from_base_key": "MonitoredVehicleJourney_MonitoredCall_Extensions_Distances_CallDistanceAlongRoute",
        "dist_to_next_stop_key": u'MonitoredVehicleJourney_MonitoredCall_Extensions_Distances_DistanceFromCall',
        "timestamp_key": "RecordedAtTime",
        "join_key": "MonitoredVehicleJourney_FramedVehicleJourneyRef_DatedVehicleJourneyRef"
    }
    globals().update(to_update)
    
def set_back_keys():
    to_update = {
        "lat_key": "latitude",
        "lng_key": "longitude",
        "bearing_key": "bearing",
        "direction_key": "direction_id",
        "progress_key":  "progress",
        "line_key": "route_id",
        "dist_from_base_key": "dist_along_route",
        "dist_to_next_stop_key": "dist_from_stop",
        "timestamp_key": "timestamp",
        "join_key": "trip_id",
    }
    globals().update(to_update)

In [19]:
set_back_keys()

In [22]:
set_live_keys()

In [23]:
lat_key

'MonitoredVehicleJourney_VehicleLocation_Latitude'

In [28]:
from datetime import timedelta
def run_simulation(df):
    """
    :param df -- **the merged dataframe (on trips.txt)**
    """
    # this is idempotent
    set_back_keys()
    base_time = df[timestamp_key].min()
    prev_time = base_time
    for i in xrange(1000):
        if not (i % 10):
            print "{} seconds elapsed".format(i)
        delta = timedelta(seconds=i)
        current_time = base_time + delta
        cur_info = df[(df[timestamp_key] == current_time) & (df[line_key] == "M116")]
        if cur_info.empty:
            continue
        update_route_measurements(cur_info)
        for route, kf_tuple in measurements_by_ref.iteritems():
            measurements_by_ref[route] = update_model(kf_tuple)

    return "Done"

In [29]:
measurements_by_ref = {}
# merged = parse_historical_input("hello")
run_simulation(merged)

0 seconds elapsed
10 seconds elapsed
20 seconds elapsed
30 seconds elapsed
40 seconds elapsed
50 seconds elapsed
60 seconds elapsed
70 seconds elapsed
80 seconds elapsed
90 seconds elapsed
100 seconds elapsed
110 seconds elapsed
120 seconds elapsed
130 seconds elapsed
140 seconds elapsed
150 seconds elapsed
160 seconds elapsed
170 seconds elapsed
180 seconds elapsed
190 seconds elapsed
200 seconds elapsed
210 seconds elapsed
220 seconds elapsed
230 seconds elapsed
240 seconds elapsed
250 seconds elapsed
260 seconds elapsed
270 seconds elapsed
280 seconds elapsed
290 seconds elapsed
300 seconds elapsed
310 seconds elapsed
320 seconds elapsed
330 seconds elapsed
340 seconds elapsed
350 seconds elapsed
360 seconds elapsed
370 seconds elapsed
380 seconds elapsed
390 seconds elapsed
400 seconds elapsed
410 seconds elapsed
420 seconds elapsed
430 seconds elapsed
440 seconds elapsed
450 seconds elapsed
460 seconds elapsed
470 seconds elapsed
480 seconds elapsed
490 seconds elapsed
500 seconds

'Done'

In [12]:
import pandas as pd
def parse_historical_input(filename):
    set_back_keys()
    df = pd.read_csv("bus_time_20150128.csv")
    trips = pd.read_csv("trips.txt")
    merged = df.merge(trips, on="trip_id")
    merged[timestamp_key] = pd.to_datetime(merged[timestamp_key], infer_datetime_format=True)
    return merged

In [83]:
vincenty((40.799888, -73.94462),
(40.799717, -73.944213)).miles

0.024386582385034945

In [30]:
kt = measurements_by_ref.get(measurements_by_ref.keys()[0])

In [33]:
sum(kt.errors)

0.9744439857832262